In [87]:
from tqdm import tqdm
from collections import Counter, defaultdict
import random
from copy import copy, deepcopy
import numpy as np
from dataclasses import dataclass, field

In [ ]:
import heapq
from dataclasses import dataclass, field

@dataclass
class Edge:
    src: int
    dst: int
    weight: int
        
@dataclass
class Graph:
    V: set
    v_to_edgelist: dict

In [3]:
FILE_NAME="dijkstraData.txt"

In [1]:
! ls -lh

total 29952
-rw-r--r--  1 Alexander  staff   498K Apr 22 02:05 g1.txt
-rw-r--r--  1 Alexander  staff   498K Apr 22 02:05 g2.txt
-rw-r--r--  1 Alexander  staff   498K Apr 22 02:05 g3.txt
-rw-r--r--  1 Alexander  staff    13M Apr 22 02:05 large.txt
-rw-r--r--  1 Alexander  staff   4.0K Apr 22 02:04 template-Copy1.ipynb


In [2]:
with open("g1.txt") as file:
    a = file.read().splitlines()

In [6]:
num_v, num_e = (int(x) for x in a[0].split(' '))
num_v, num_e

(1000, 47978)

In [7]:
a[1:6]

['1 14 6', '1 25 47', '1 70 22', '1 82 31', '1 98 17']

In [20]:
V = set()
v_to_edgelist = {}
for x in a[1:]:
    x = x.split(' ')
    src = int(x[0])
    dst = int(x[1])
    weight = int(x[2])
    edge = Edge(src=src, dst=dst, weight=weight)
    V.add(src)
    if src in v_to_edgelist:
        v_to_edgelist[src].append(edge)
    else:
        v_to_edgelist[src] = [edge]

In [21]:
G = Graph(V, v_to_edgelist)

# Floyd-Warshall (APSP)

In [16]:
def Floyd_Warshall_APSP(G):
    num_v = len(G.V)
    A = np.full((num_v+1, num_v+1, num_v+1), np.inf)
    for i in range(num_v):
        A[i,i,0] = 0
    for edges in G.v_to_edgelist.values():
        for edge in edges:
            A[edge.src, edge.dst, 0] = edge.weight
    
    for k in tqdm(range(1, num_v+1)): # num of allowed edges
        for i in range(1, num_v+1): # from each src
            for j in range(1, num_v+1): # to each dst
                A[i,j,k] = min(A[i,j,k-1], A[i,k,k-1] + A[k,j,k-1])
    for i in range(1, num_v+1):
        if A[i,i,-1] < 0:
            return None
    return A[:,:,-2]

In [17]:
def read_graph(file_name):
    with open(file_name) as file:
        a = file.read().splitlines()
    num_v, num_e = (int(x) for x in a[0].split(' '))
    V = set()
    v_to_edgelist = {}
    for x in a[1:]:
        x = x.split(' ')
        src = int(x[0])
        dst = int(x[1])
        weight = int(x[2])
        edge = Edge(src=src, dst=dst, weight=weight)
        V.add(src)
        V.add(dst)
        if src in v_to_edgelist:
            v_to_edgelist[src].append(edge)
        else:
            v_to_edgelist[src] = [edge]
    G = Graph(V, v_to_edgelist)
    return G

In [18]:
%load_ext jupyternotify

The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [19]:
%%notify
for file in ['g1.txt', 'g2.txt', 'g3.txt']:
    print(file)
    G = read_graph(file)
    A = Floyd_Warshall_APSP(G)
    if A is not None:
        print(A.min())
    else:
        print('cycle')

g1.txt


100%|██████████| 1000/1000 [22:49<00:00,  1.37s/it]


cycle
g2.txt


100%|██████████| 1000/1000 [24:01<00:00,  1.44s/it]


cycle
g3.txt


100%|██████████| 1000/1000 [24:01<00:00,  1.44s/it]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

<IPython.core.display.Javascript object>

In [21]:
A.min()

-19.0

In [160]:
def Dijkstra_SSSP(G, start_v):
    G = deepcopy(G)
    D = defaultdict(lambda: np.inf)
    curr_v = start_v
    G.V.remove(curr_v)
    D[curr_v] = 0
    
    heap = []
    while G.V:
        if curr_v in G.v_to_edgelist:
            edges = G.v_to_edgelist[curr_v]
            for edge in edges:
                dist = min(D[edge.dst], D[edge.src] + edge.weight)
                D[edge.dst] = dist
                heapq.heappush(heap, (dist, edge.dst))
        
        if len(heap) == 0: break
        dist, dst = heapq.heappop(heap)
        flag = False
        while dst not in G.V: # it will stop when finds dst that is in G.V (not explored)
            if len(heap) == 0: 
                flag = True
                break
            dist, dst = heapq.heappop(heap) # clean heap from edges whos v are already in D
        if flag: break
        D[dst] = dist
        G.V.remove(dst)
        curr_v = dst
    D = dict(D)
    return D

In [161]:
G = read_graph('test_dijkstra.txt')
A = Dijkstra_SSSP(G, 0)
A

{0: 0, 1: 2, 3: 1, 2: 5}

In [145]:
G = read_graph('test_bellman.txt')
A = Dijkstra_SSSP(G, 0)
A

{0: 0, 1: 2, 2: 3, 3: 4, 4: 6}

In [157]:
def Bellman_Ford_SSSP(G):
    num_v = len(G.V)
    v_to_in_edges = {}
    # assume zero is source vertex
    v_to_in_edges[0] = []
    for v, edges in G.v_to_edgelist.items():
        for edge in edges:
            dst = edge.dst
            if dst in v_to_in_edges:
                v_to_in_edges[dst].append(edge)
            else:
                v_to_in_edges[dst] = [edge]
    A = np.zeros((num_v+1, num_v)) # num_allowed_edges [0-n] x numerated verted 
    for v in G.V:
        A[0,v] = np.inf
    A[0,0] = 0
    for i in tqdm(range(1, num_v+1)): # num allowed edges
        for v in G.V:
            paths_to_v = []
            for in_edge in v_to_in_edges[v]:
                paths_to_v.append(A[i-1, in_edge.src] + in_edge.weight)
            if len(paths_to_v) == 0:
                A[i,v] = A[i-1,v]
            else:
                A[i,v] = min(A[i-1,v], min(paths_to_v))
    for v in G.V:
        if A[num_v-1, v] != A[num_v, v]:
            return None
    return A[-2,:]

In [126]:
G = read_graph('test_bellman.txt')
A = Bellman_Ford_SSSP(G)
A

100%|██████████| 5/5 [00:00<00:00, 35424.86it/s]


array([0., 2., 3., 4., 6.])

In [166]:
def Johnson_APSP(G):
    # assume 0 is not in G.V
    # 1. add edges from 0 to each v with weight 0
    G = deepcopy(G)
    G_zero = deepcopy(G)
    G_zero.v_to_edgelist[0] = []
    for v in G_zero.V:
        G_zero.v_to_edgelist[0].append(Edge(src=0, dst=v, weight=0))
    G_zero.V.add(0)
    # 2. calc ps with Bellman_Ford_SSSP or halt
    A = Bellman_Ford_SSSP(G_zero) # distances from 0 to each vertex or None if cycle
    if A is None:
        return None
    # 3. reweight
    for edges in G.v_to_edgelist.values():
        for edge in edges:
            edge.weight = edge.weight + A[edge.src] - A[edge.dst]
    # 4. for each v run Dijkstra
    res = np.inf
    for src in tqdm(G.V):
        D = Dijkstra_SSSP(G, src)
        for dst, path_len in D.items():
            # 5. reweight
            res = min(res, path_len - A[src] + A[dst])
    return res

In [165]:
G = read_graph('test_bellman.txt')
A = Johnson_APSP(G)
A

100%|██████████| 5/5 [00:00<00:00, 2902.23it/s]


-3.0

In [ ]:
for file in ['g1.txt', 'g2.txt', 'g3.txt']:
    print(file)
    G = read_graph(file)
    A = Johnson_APSP(G)
    if A is not None:
        print(A.min())
    else:
        print('cycle')

g1.txt


100%|██████████| 1001/1001 [00:58<00:00, 17.14it/s]


cycle
g2.txt


100%|██████████| 1001/1001 [00:59<00:00, 16.92it/s]


cycle
g3.txt


 10%|█         | 103/1000 [01:24<12:27,  1.20it/s]